# Supplementary Source Code 1 - Crop Imaging on CropQuant
This Jupyter notebook describes the crop imaging script designed for the single-board computer, Raspberry Pi, on CropQuant workstation. The algorithm below has been optimised for Pi computers and unnecessary libraries such as SimpleCV has been removed due to power consumption issues. 

This script will capture time-lapse image series through the CropQuant hardware and remove low-quality images if they are dark, out-of-focus, or taken in low lighting conditions. The script will provide a GUI for users to enter experiment related details and imaging frequency. 

## Please note that:
1. A Jupyter notebook is ONLY used for demonstrating the algorithm, NOT for batch processing big crop image series. 
2. The performance of the algorithm provided for the CropQuant breeding project could be varied due to an end-user's computing resources such as physical memory and CPU speed. 
3. Users might encounter issues with specific libraries if they have not installed properly or not been optimised for Linux-based operating systems. 
4. For all Scikit-image or Scikit-learning functions, authors recommend users to preinstall the latest open Anaconda Python distribution. 


In [1]:
#################################################################
#                                                               #
# The imaging script is designed for the joint JIC and EI       #
# wheat growth research project and the CropQuant project.      #
# The script needs to be operated on a Raspberry Pi computer.   #
#                                                               #
#    Author: Dr Ji Zhou (周济), ji.zhou@earlham.ac.uk or         #
#                        ji.zhou@jic.ac.uk                      #
#    Date: 1st November 2015, V0.1 10th May 2015                #
#                                                               #
#    Stable version: 1.05 on EI internal Github and Bitbcuket   #
#    https://bitbucket.org/Phenomics/cropquant                  #
#    picamera version: 1.05                                     #
#                                                               #
#    Software license: the bespoke CropQuant Software License   #
#    The algorithm below is running on CropQuant workstations   #
#    developed by Dr Zhou and his team, which shall only be     #
#    used for crop research and JIC/EI wheat research           #
#                                                               #
#    Changes: 1) Add result folder                              #
#             2) Change the time stamp                          #
#             3) Rectify USB memory pen issue                   #
#             4) Add file size detection                        #
#             5) Remove simpleCV to decrease computational      #
#                complexity running on Pi computers             #
#                                                               #
#    Note: from V1.05 onwards, source code has been modulated   #
#    and integrated in the CropMonitor control system in order  # 
#    to provide a unified API for CQ monitoring                 #
#    see CropQuant Github                                       #
#                                                               #
#################################################################

In [3]:
# General imports
import time
import picamera
import math
from shutil import move

# Import IP socket reader
import socket
import fcntl
import struct

In [5]:
# All used functions 
#################################################################
# Start the main imaging procedure                              #
#################################################################
def get_ip_address(ifname):
    """ IP address based on socket information """
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    return socket.inet_ntoa(fcntl.ioctl(
            s.fileno(),
            0x8915,  # SIOCGIFADDR
            struct.pack('256s', ifname[:15])
            )[20:24])


def get_date_time():
    """ Use raspberry pi time, local time, as the time point """
    timestamp_local = time.localtime(time.time())
    return timestamp_local


def wait():
    """ Calculate the delay to the start of the next hour """
    # (1) dynamic input based on GUI
    # next_image =(datetime.now()+timedelta(hour=0)).replace(minute=0,second=10,microsecond=0)
    # delay = (next_image - datetime.now()).seconds
    # (2) Use the pi2/3 to calculate the 20 minutes interval
    time.sleep(1200) 
    # Using seconds as pi computers are now synchronise with an infield DHCP server
    # three images per hour, 1200 seconds, 48 images per day
    # Roughly 336 images will be generated during a week's image acquistion


def fileTransfer(imagePath, resultPath, imagename):
    """ File transfer function """
    from shutil import copyfile
    imageDir = imagePath + imagename
    resultDir = resultPath + '/' + imagename
    copyfile(imageDir, resultDir)


# from SimpleCV import Image
# Remove SimpleCV for v1.0 onwards - see to in-field trait analysis
def show_entry_fields():
    """ Use Tkinter as the shell to encapsulate the imaging process """
    # If issues due to hardware during the season, errors can be enclosed
    # Error/warning message can be traced through the enclosed libraries
    
    # Determine the time stamp of the image
    from datetime import datetime, timedelta

    #-- Getting the date and time
    # Time and date need to be set up on rapsberry pi first!
    timestamp = get_date_time()
    curYear = timestamp[0]
    curMonth = timestamp[1]
    curDay = timestamp[2]
    curHour = timestamp[3]
    curMin = timestamp[4]
    
    print("Experiment name: %s\nImaging device: %s\nBiological replicates: %s\n Days: %s" %
          (E1.get(), E2.get(), E3.get(), E4.get()))
    # print curDay, curMonth, curYear
    CurrentDir = os.getcwd()

    ResultFolder = 'Experiment_%d' %curDay + '-%d' %curMonth + '-%d' %curYear
    
    # If usb card has been inserted
    if os.path.exists('/media/FIELDWHEAT/FieldTrial'):
        ResultDirectory_BackUp = '/media/FIELDWHEAT/' + ResultFolder
        ResultDirectory_IMG = CurrentDir + '/' + ResultFolder
    else:
        ResultDirectory_BackUp = '/home/pi/Desktop/Backup/' + ResultFolder
        # The USB has not been inserted
        ResultDirectory_IMG = CurrentDir + '/' + ResultFolder

    if not os.path.exists(ResultDirectory_BackUp):
        os.makedirs(ResultDirectory_BackUp)

    if not os.path.exists(ResultDirectory_IMG):
        os.makedirs(ResultDirectory_IMG)
    # Finish creating the result folder

    # set how many shots to take in the field
    for i in range(int(E4.get()) * 24 * 3):
        # In order not to exceed the memory limit
        # The data will be backed up in a usb memory pen
        
        # Establish a day folder that contains all images captured in a particular day
        # In DD_MM_YY
        DayResultFolder = 'Date_%d'%curDay + '_%d'%curMonth + '_%s'%str(curYear)[-2:]
        DayResultDirectory_BackUp = ResultDirectory_BackUp + '/' + DayResultFolder
        DayResultDirectory_IMG = ResultDirectory_IMG + '/' + DayResultFolder
        # DayResultDirectory_Bad = DayResultDirectory_BackUp + '/Discarded'
        
        # Detect whether or not the day folder has been created
        if not os.path.exists(DayResultDirectory_BackUp):
            os.makedirs(DayResultDirectory_BackUp)
        # if not os.path.exists(DayResultDirectory_Bad): # Bad image folder
        #    os.makedirs(DayResultDirectory_Bad)
        if not os.path.exists(DayResultDirectory_IMG):
            os.makedirs(DayResultDirectory_IMG)
            # Create the day folder to store captured images

        # Start to form an picamera object
        with picamera.PiCamera() as camera:
            
            #-- Camera setting --#
            camera.brightness = 45
            # Give the relative low lighting in the chamber
            camera.saturation = 5
            camera.contrast = 5
            camera.sharpness = 10 
            # Increase sharpness
            
            # Set up white balance
            g = camera.awb_gains
            camera.awb_mode = 'auto' # Switch to 'off' can have direct control
            camera.awb_gains = g
            # Wait for the automatic gain control to settle
            
            # Define shutter speed and exposure mode
            camera.shutter_speed = camera.exposure_speed 
            # use fixed expose value
            camera.exposure_mode = 'auto'
            
            # Define resolution
            camera.resolution = (2592, 1944)
            camera.framerate = 24 # 24 frames
            # Longer exposure time gives better imagr quality
            #-- Finish setting up Camera --#
            
            # Imaging starts
            camera.start_preview()
            time.sleep(10) # Stablise the camera for imaging
            
            # Set up image name
            # Read teh imaging date/time
            timestamp = get_date_time()
            curYear = timestamp[0]
            curMonth = timestamp[1]
            curDay = timestamp[2]
            curHour = timestamp[3]
            curMin = timestamp[4]
            # Form the image filename
            filename_tmp = E1.get() + '_Rep%s'%E3.get() + '_%s'%E2.get() + '_ID-%02d'%(i+1)
            date_tmp = '_Date-%d'%curDay + '-%d'%curMonth + '-%d'%curYear + '_%d'%curHour + '-%d'%curMin
            filename = filename_tmp + date_tmp + '.jpg'
            # Taking an image
            camera.capture(filename)
            
            # Print out how many has been imaged
            # This will be displayed on the command line interface, which will be seen either
            # through VNC or the CropMonitor system
            if (i == 0):
                print('The %d st run: ' %(i+1) + 'Captured image %s' %filename)
            elif (i == 1):
                print('The %d nd run: ' %(i+1) + 'Captured image %s' %filename)
            # The third run onwards
            if (i >= 2):
                print('The %d th run: ' %(i+1) + 'Captured image %s' %filename)

            # Test the size of the file, remove SimpleCV for modulating the functions
            # img = img.edges()
            file_stat = os.stat(filename)
            filesize = 1.0 * file_stat.st_size/(1024**2) 
            print round(filesize, 3)
            
            if filesize >= 3.0: # file has a size of 3.0MB, 5MB RGB or NoIR cameras
                # Organise the image path for file transfer
                imagePath = CurrentDir + '/'
                fileTransfer(imagePath, DayResultDirectory_BackUp, filename)
                # Move the captured image file to the result folder
                move(CurrentDir + '/' + filename, DayResultDirectory_IMG + '/' + filename)
            else: 
                # Organise the image path for file transfer
                imagePath = CurrentDir + '/'
                fileTransfer(imagePath, DayResultDirectory_Bad, filename)
                # Delete the bad image file
                os.remove(filename)
            # Exception handling will terminate the imaging while sleeping!!!

        # camera.stop_preview()
        # Finish powering the camera
        wait() # Wait for the next shot
        # Finish looping through the system

    # Break out the loop, close up the preview
    camera.stop_preview()
    # The GUI instance will regain the control
    

In [10]:
#################################################################
# Use Tk as the main function                                   #
#################################################################

# Added for capturing user input
# GUI panel to accept input
from Tkinter import *
import os

CurrentDirectory = os.getcwd()
#print CurrentDirectory
CurrentExp = CurrentDirectory.split('/')[-1]

#-- Getting the IP Address
# This function will read the ip address from wifi dongle
# and prepopulate the last two digits of the IP address onto the input GUI
IP_Address = get_ip_address('wlan0') #... Onsite setting ...#
# IP_Address = get_ip_address('eth0') # if the user use Ethernet to run the CropQuant
# default static in-field IP Address is "169.254.2.18" for offsite setting
Device_ID = IP_Address.split('.')[-1] # Automatically recognise the CropQuant workstation ID
#-- Read the IP from ethernet port


top = Tk(baseName = "Infield_Wheat_Monitoring_System")
top.title("JIC & TGAC Wheat CropQuant")
L1 = Label(top, text="Experiment name").grid(row=1)
L2 = Label(top, text="Imaging device ID").grid(row=2)
L3 = Label(top, text="Biological replicates").grid(row=3)
L4 = Label(top, text="Duration (in days)").grid(row=4)

E1 = Entry(top, bd = 5)
E1.insert(0, CurrentExp[:-3])
E2 = Entry(top, bd = 5)
E2.insert(0, int(Device_ID[-2:]))
E3 = Entry(top, bd = 5)
E4 = Entry(top, bd = 5)

E1.grid(row=1, column=1)
E2.grid(row=2, column=1)
E3.grid(row=3, column=1)
E4.grid(row=4, column=1)

Button(top, text='Quit', command=top.quit).grid(row=5, column=1, sticky=W, pady=4)
Button(top, text='Start', command=show_entry_fields).grid(row=5, column=0, sticky=W, pady=4)

# Start the main loop for imaging
# Encapsulate the imaging procedure in the GUI instance
top.mainloop()
# Finish the GUI input, Ctrl+C can force the process to stop. 
